# Calcul du contraste lésion / voisinage sur T1

### Ce code calcule le contraste d’une lésion par rapport à son environnement immédiat dans une image T1. Il lit le masque de la lésion et l’image T1, dilate le masque pour définir un voisinage autour de la lésion, puis calcule la moyenne d’intensité dans la lésion et dans le voisinage. Enfin, il renvoie le contraste sous forme de ratio moyenne lésion / moyenne voisinage, indiquant si la lésion est plus brillante ou plus sombre que son environnement.

In [7]:
import SimpleITK as sitk
import numpy as np
import csv

# -----------------------------
# 1) Chemins des fichiers
# -----------------------------
mask_path = "../../27919209/MSLesSegDataset/train/P1/T1/P1_T1_MASK.nii.gz"
t1_path   = "../../27919209/MSLesSegDataset/train/P1/T1/P1_T1_T1.nii.gz"

# -----------------------------
# 2) Lecture des images
# -----------------------------
t1_img   = sitk.Cast(sitk.ReadImage(t1_path), sitk.sitkFloat64)
mask_img = sitk.Cast(sitk.ReadImage(mask_path) > 0, sitk.sitkUInt8)

t1_array   = sitk.GetArrayFromImage(t1_img)
mask_array = sitk.GetArrayFromImage(mask_img)

# -----------------------------
# 3) Identifier les lésions distinctes
# -----------------------------
mask_labels = sitk.ConnectedComponent(mask_img)
mask_labels = sitk.RelabelComponent(mask_labels)
labels_array = sitk.GetArrayFromImage(mask_labels)

num_lesions = int(labels_array.max())
print(f"Nombre de lésions détectées : {num_lesions}")

# -----------------------------
# 4) Calcul contraste pour chaque lésion
# -----------------------------
results = []

# Définir un noyau 3D pour dilatation (3x3x3)
# Adapter la taille du noyau en fonction de la taille des lesions (faire des essais)
kernel = sitk.sitkBall
radius = (1, 1, 1)  # dilatation sur 1 voxel dans chaque direction

for lesion_id in range(1, num_lesions + 1):
    lesion_mask_bool = (labels_array == lesion_id)
    lesion_mask_sitk = sitk.GetImageFromArray(lesion_mask_bool.astype(np.uint8))
    lesion_mask_sitk.CopyInformation(t1_img)

    # Moyenne lésion
    lesion_voxels = t1_array[labels_array == lesion_id]
    lesion_mean = round(lesion_voxels.mean(), 1)

    # Dilatation pour définir le voisinage
    dilated_mask = sitk.BinaryDilate(lesion_mask_sitk, radius, kernelType=kernel)
    dilated_array = sitk.GetArrayFromImage(dilated_mask)

    # Voisinage = dilatation - lésion
    neighborhood_mask = (dilated_array > 0) & (labels_array != lesion_id)
    neighborhood_voxels = t1_array[neighborhood_mask]

    if neighborhood_voxels.size > 0:
        neighborhood_mean = round(neighborhood_voxels.mean(), 1)
        contrast = round(lesion_mean / neighborhood_mean, 2)
    else:
        neighborhood_mean = ""
        contrast = ""

    results.append({
        "patient": "P1",
        "timepoint": "T1",
        "lesion_id": lesion_id,
        "lesion_mean": lesion_mean,
        "neighborhood_mean": neighborhood_mean,
        "contrast_lesion_neighborhood": contrast
    })

# -----------------------------
# 5) Sauvegarder CSV
# -----------------------------
csv_file = "P1_T1_T1_lesions_contrast_simpleitk.csv"
keys = ["patient", "timepoint", "lesion_id", "lesion_mean", "neighborhood_mean", "contrast_lesion_neighborhood"]

with open(csv_file, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writeheader()
    for row in results:
        writer.writerow(row)

print(f"CSV sauvegardé : {csv_file}")


Nombre de lésions détectées : 18
CSV sauvegardé : P1_T1_T1_lesions_contrast_simpleitk.csv


### Trouver un moyen de savoir quand la lesion est presente dans une zone, cette zone correspond a quelle taille de la lésion

### compter le nombre de pixels sur qui appartiennet a la lesion pour chaque label aussi (pour les 3 (T1, T2, FLAIR))

### rgd mom tch
